In [1]:
from colabcode import ColabCode
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from fastapi import FastAPI, UploadFile, File, HTTPException, Depends
from fastapi.middleware.cors import CORSMiddleware
from PIL import Image
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
!ngrok authtoken 2cNVnDEt0N2Cyhe07s7wvLZcH7H_4BeSM2pxWXtvgqxePsfzs

Authtoken saved to configuration file: C:\Users\madhu\AppData\Local/ngrok/ngrok.yml


In [3]:
from tensorflow.keras.models import load_model

# Provide the path to your saved model file
model_path = "./MyModel.h5"

# Load the model
MODEL = load_model(model_path)
cc = ColabCode(port=12000, code=False)

CLASS_NAMES = ['balloon flower', 'black-eyed susan', 'foxglove', 'frangipani', 'jasmine', 'lotus lotus', 'orange hibiscus', 'orange marigold', 'oxeye daisy', 'pink hibiscus', 'pink rose', 'red hibiscus', 'redRose', 'stemless gentian', 'sunflower', 'thorn apple', 'water lily', 'yellow hibiscus', 'yellow marigold', 'yellow rose']

app = FastAPI()

def get_model():
    return MODEL

# Use the CORS middleware to allow all origins, methods, and headers
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
async def read():
    return "I am live"

def read_file_as_image(data) -> np.ndarray:
    try:
        image = Image.open(BytesIO(data))
        # Resize the image to match the expected input shape (240, 240)
        image = image.resize((250, 250))
        image = np.array(image)
        return image
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error reading image: {str(e)}")

@app.post("/predict")
async def predict(
    file: UploadFile = File(..., content_type="image/jpeg"),
    model: tf.keras.Model = Depends(get_model)
):
    try:
        image = read_file_as_image(await file.read())
        image_batch = np.expand_dims(image, 0)
        prediction = model.predict(image_batch)
        predicted_class = CLASS_NAMES[np.argmax(prediction[0])]
        confidence = float(np.max(prediction[0]))
        return {
            "class": predicted_class,
            "confidence": confidence
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://8f39-103-18-165-128.ngrok-free.app" -> "http://localhost:12000"


INFO:     Started server process [6720]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60930 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:60930 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:60930 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:60930 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:60940 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60940 - "GET /openapi.json HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 966ms/step
INFO:     127.0.0.1:60955 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [6720]
